In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# to use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import sys
sys.path.append('..')

In [4]:
from mgtbench.loading.dataloader import load_attribution_topic
# Humanities Social_sciences
data = load_attribution_topic('Social_sciences')

Model-based Attribution

In [5]:
from mgtbench import AutoDetector, AutoExperiment
nclass = len(set(data['train']['label']))
model= '/data1/zzy/roberta-base'
metric = AutoDetector.from_detector_name('LM-D',
                                        model_name_or_path=model,
                                        num_labels=nclass,
                                        use_metric=True
                                        )
experiment = AutoExperiment.from_experiment_name('supervised',detector=[metric])
experiment.load_data(data)

config = {
    'need_finetune': True,
    'need_save': False,
    'epochs': 2,
    'batch_size': 64,
    'lr':5e-6,
    'disable_tqdm': True
    }
res = experiment.launch(**config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data1/zzy/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512
Calculate result for each data point
Running prediction of detector LM-D
False
using metric trainer


/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

Step,Training Loss
30,5.938900
60,4.939800
90,4.555600
120,4.454800
150,4.242400
180,4.161800
210,4.142500
240,4.057500
270,3.957400


Fine-tune finished
Predict training data
Predict testing data
Run classification for results
[[358   0   4   1   1   0]
 [  0 253   9  42  37  15]
 [  5   7 281  25   9  24]
 [  0  35  26 266  28   3]
 [  2  55   8  27 224  34]
 [  0  41  24   4  39 258]]
[[1406    0   16    1    1    0]
 [   2 1031   26  158  157   58]
 [  37   24 1180   67   55   74]
 [   3  134   60 1042  162   29]
 [   5  195   63  140  911  124]
 [   8  125   83   27  140 1039]]


In [6]:
res

[DetectOutput(name='test_pred', train=None, test=Metric(acc=0.7645687645687645, precision=0.7660343494346146, recall=0.7645687645687645, f1=0.7647463869086015, auc=-1.0, conf_m=array([[358,   0,   4,   1,   1,   0],
        [  0, 253,   9,  42,  37,  15],
        [  5,   7, 281,  25,   9,  24],
        [  0,  35,  26, 266,  28,   3],
        [  2,  55,   8,  27, 224,  34],
        [  0,  41,  24,   4,  39, 258]]))),
 DetectOutput(name='train_pred', train=None, test=Metric(acc=0.7700104858441105, precision=0.7700292688758228, recall=0.7700104858441105, f1=0.769749548423765, auc=-1.0, conf_m=array([[1406,    0,   16,    1,    1,    0],
        [   2, 1031,   26,  158,  157,   58],
        [  37,   24, 1180,   67,   55,   74],
        [   3,  134,   60, 1042,  162,   29],
        [   5,  195,   63,  140,  911,  124],
        [   8,  125,   83,   27,  140, 1039]])))]

metric-based attribution

multi-metric-based attribution

First, you may generate different metrics and save them.

In [ ]:
from mgtbench import AutoDetector, AutoExperiment
import numpy as np
# log-likelihood detector
model_name_or_path = '/data1/models/Llama-2-7b-chat-hf'
metric = AutoDetector.from_detector_name('rank',
                                         model_name_or_path=model_name_or_path)
lls_train = metric.detect(data['train']['text'],log=True)
lls_test = metric.detect(data['test']['text'],log=True)
X_train = np.array(lls_train).reshape(-1,1)
X_test = np.array(lls_test).reshape(-1,1)
data_dict = {
    "X_train": X_train,
    "y_train": data['train']['label'],
    "X_test": X_test,
    "y_test": data['test']['label']
}
import pickle

# Save to a pickle file
folder = 'kdd'
output_file = os.path.join(folder, "rank_stem.pkl")
with open(output_file, "wb") as f:
    pickle.dump(data_dict, f)

print(f"Data saved to {output_file}")

Then, feel free to load and concat them for inference

In [ ]:
import json
folder = 'kdd_full'
TOPICS = ['STEM', 'Humanities', 'Social_sciences']
final_result = {}

for topic in TOPICS:
    # Load the pickle files
    topic = topic.lower()
    entropy = pickle.load(open(os.path.join(folder, f"entropy_{topic}.pkl"), "rb"))
    log_rank = pickle.load(open(os.path.join(folder, f"rank_{topic}.pkl"), "rb"))
    ll = pickle.load(open(os.path.join(folder, f"ll_{topic}.pkl"), "rb"))
    fast = pickle.load(open(os.path.join(folder, f"fast-detectgpt_{topic}.pkl"), "rb"))
    bino = pickle.load(open(os.path.join(folder, f"binoculars_{topic}.pkl"), "rb"))

    # Extract training and testing sets from each file
    X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy = tuple(entropy.values())
    X_train_logrank, y_train_logrank, X_test_logrank, y_test_logrank = tuple(log_rank.values())
    X_train_ll, y_train_ll, X_test_ll, y_test_ll = tuple(ll.values())
    X_train_fast, y_train_fast, X_test_fast, y_test_fast = tuple(fast.values())
    X_train_bino, y_train_bino, X_test_bino, y_test_bino = tuple(bino.values())


    # Function to evaluate performance for a given set of features
    def evaluate_performance(X_train, X_test, y_train, y_test):
        model = LogisticRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')  # Using weighted average for multi-class support
        return accuracy, f1

    import xgboost as xgb
    from sklearn.metrics import accuracy_score, f1_score
    import numpy as np

    # Function to evaluate performance and get feature importance
    def evaluate_performance_xgboost(X_train, X_test, y_train, y_test):
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        model.fit(X_train, y_train)
        
        # Predicting on test data
        y_pred = model.predict(X_test)
        
        # Evaluate performance metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')  # Using weighted average for multi-class support
        
        # Get feature importances (most significant features)
        feature_importances = model.feature_importances_
        
        print('ll\tlogrank\tentropy\tfast\tbino')
        print(feature_importances)
        
        # Get the index of the most important feature
        most_important_feature_index = np.argmax(feature_importances)
        
        # Get the importance score of the most important feature
        most_important_feature_score = feature_importances[most_important_feature_index]
        
        return accuracy, f1, most_important_feature_index, most_important_feature_score, feature_importances.tolist()

    # Example usage

    # Track performance for different combinations of features
    results = {}
    accuracy_concat1, f1_concat1 = evaluate_performance(X_train_ll, X_test_ll, y_train_ll, y_test_ll)
    results['ll'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    accuracy_concat1, f1_concat1 = evaluate_performance(X_train_logrank, X_test_logrank, y_train_logrank, y_test_logrank)
    results['log_rank'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    accuracy_concat1, f1_concat1 = evaluate_performance(X_train_entropy, X_test_entropy, y_train_entropy, y_test_entropy)
    results['entropy'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    accuracy_concat1, f1_concat1 = evaluate_performance(X_train_fast, X_test_fast, y_train_fast, y_test_fast)
    results['fast'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    accuracy_concat1, f1_concat1 = evaluate_performance(X_train_bino, X_test_bino, y_train_bino, y_test_bino)
    results['bino'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    # # Case 1: Using only one feature
    # X_train_concat1 = X_train_entropy
    # X_test_concat1 = X_test_entropy
    # accuracy_concat1, f1_concat1 = evaluate_performance(X_train_concat1, X_test_concat1, y_train_entropy, y_test_entropy)
    # results['entropy'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    # accuracy_concat1, f1_concat1 = evaluate_performance(X_train_fast, X_test_fast, y_train_fast, y_test_fast)
    # results['fast'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    # accuracy_concat1, f1_concat1 = evaluate_performance(X_train_bino, X_test_bino, y_train_bino, y_test_bino)
    # results['bino'] = {'accuracy': accuracy_concat1, 'f1': f1_concat1}

    # # Case 2: Using entropy + log_rank features
    # X_train_concat2 = np.concatenate((X_train_concat1, X_train_logrank), axis=1)
    # X_test_concat2 = np.concatenate((X_test_concat1, X_test_logrank), axis=1)
    # accuracy_concat2, f1_concat2 = evaluate_performance(X_train_concat2, X_test_concat2, y_train_entropy, y_test_entropy)
    # results['entropy + log_rank'] = {'accuracy': accuracy_concat2, 'f1': f1_concat2}

    # # Case 2: Using ll + log_rank features
    # X_train_concat2 = np.concatenate((X_train_ll, X_train_logrank), axis=1)
    # X_test_concat2 = np.concatenate((X_test_ll, X_test_logrank), axis=1)
    # accuracy_concat2, f1_concat2 = evaluate_performance(X_train_concat2, X_test_concat2, y_train_ll, y_test_ll)
    # results['ll + log_rank'] = {'accuracy': accuracy_concat2, 'f1': f1_concat2}

    # # Case 2: fast + bino features
    # X_train_concat2 = np.concatenate((X_train_fast, X_train_bino), axis=1)


    # # Case 3: Using entropy + log_rank + ll features
    # X_train_concat3 = np.concatenate((X_train_concat2, X_train_ll), axis=1)
    # X_test_concat3 = np.concatenate((X_test_concat2, X_test_ll), axis=1)
    # accuracy_concat3, f1_concat3 = evaluate_performance(X_train_concat3, X_test_concat3, y_train_entropy, y_test_entropy)
    # results['entropy + log_rank + ll'] = {'accuracy': accuracy_concat3, 'f1': f1_concat3}

    # Use all
    X_train_concat_all = np.concatenate((X_train_ll, X_train_logrank, X_train_entropy, X_train_fast, X_train_bino), axis=1)
    X_test_concat_all = np.concatenate((X_test_ll, X_test_logrank, X_test_entropy, X_test_fast, X_test_bino), axis=1)
    accuracy, f1, most_important_feature, importance_score, feature_importances = evaluate_performance_xgboost(X_train_concat_all, X_test_concat_all, y_train_entropy, y_test_entropy)
    results['all'] = {'accuracy': accuracy,
                    'f1': f1,
                    'most_important_feature': int(most_important_feature),
                    'importance_score': float(importance_score), 
                    'order': ['ll', 'log_rank', 'entropy', 'fast', 'bino'],
                    'feature_importances': feature_importances
                    }
    
    final_result[topic] = results
    

# round all float to 4 decimal places
final_result = json.loads(json.dumps(final_result), parse_float=lambda x: round(float(x), 4))
json.dump(final_result, open(os.path.join(folder, "results.json"), "w"), indent=4)
    # # Print results
    # for features, scores in results.items():
    #     print(f"Performance using {features}:")
    #     print(f"  Accuracy: {scores['accuracy']:.4f}")
    #     print(f"  F1 Score: {scores['f1']:.4f}")
    #     if 'most_important_feature' in scores:
    #         print(f"  Most important feature: {scores['most_important_feature']}")
    #         print(f"  Importance score: {scores['importance_score']:.4f}")
    #     print()
